In [221]:
import xml.etree.ElementTree as ET
import copy

def isLeaf(node):
    t = True
    for child in node:
        t = False
    return t

def hasToken(node):
    return node.text.strip() != ''

def nodeIsVisited(node):
    return node.get('visited') != None

def visitNode(node):
    node.set('visited', True)

def getTag(node):
    return node.tag.replace('{http://www.srcML.org/srcML/src}', '')

tag_id = {}

def dfsPathKeep(node):
    path_stack = []
    stack = []
    visit_list = []
    
    stack.append(node)
    visited_child_count_list = []
    while (len(stack) > 0):
        currNode = stack.pop()
        # print(currNode)
        if not nodeIsVisited(currNode):
            path_stack.append(getTag(currNode))
            if hasToken(currNode):
                path_token_str = f'<{path_stack}>:{currNode.text.strip()}'
                visit_list.append(path_token_str)
            visitNode(currNode)
            visited_child_count = 0
            for child in currNode:
                if not nodeIsVisited(child):
                    stack.append(child)
                    visited_child_count += 1

            # print(visited_child_count_list, visited_child_count, visit_list[-1])
            if visited_child_count == 0:
                if len(visited_child_count_list) > 0:
                    visited_child_count_list[-1] -= 1
                    path_stack.pop()
                    # print('pop')
                while len(visited_child_count_list) > 0 and visited_child_count_list[-1] == 0:
                    visited_child_count_list = visited_child_count_list[:-2]
                    if len(visited_child_count_list) > 0:
                        visited_child_count_list[-1] -= 1
                        path_stack.pop()
                        # print('pop3')
                        
                    path_stack.pop()
                    # print('pop2')
            else:
                visited_child_count_list.append(visited_child_count)
            

    for v in visit_list:
        print(v)

def parseXML(xml_path):
    tree = ET.parse(xml_path)
    
    root = tree.getroot()
    
    dfsPathKeep(root)
    
parseXML('srcxml/juliet.java.0.test.xml')

<['unit', 'import']>:import
<['unit', 'import', 'name', 'name']>:*
<['unit', 'import', 'name', 'operator']>:.
<['unit', 'import', 'name', 'name']>:testcasesupport
<['unit', 'package']>:package
<['unit', 'package', 'name', 'name']>:CWE23_Relative_Path_Traversal
<['unit', 'package', 'name', 'operator']>:.
<['unit', 'package', 'name', 'name']>:testcases2
<['unit', 'package']>:package
<['unit', 'package', 'name', 'name']>:CWE23_Relative_Path_Traversal
<['unit', 'package', 'name', 'operator']>:.
<['unit', 'package', 'name', 'name']>:testcases
